<a href="https://colab.research.google.com/github/elviawu/mask-data/blob/main/Mask_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = res.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

print(data)

In [ ]:
# 填入欄位名稱
med_count = {}
for d in data['features']:
    county = d['properties']['county']
    if county == "":
      county = d['properties']['address'][:3]
    if county not in med_count:
         med_count[county] = 1
    else:
      med_count[county] += 1


print(med_count)
# {'台北市': 123, '新北市': 456 ...}

In [ ]:
import sqlite3
import datetime
import requests
import json

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 藥局數量，新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料

url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)
d = res.text
data = json.loads(d)

med = {}

for d in data['features']:
  city = d['properties']['county']
  if city == "":
    city = d['properties']['address'][:3]
  if city not in med:
    med[city] = 1
  else:
    med[city] += 1
for m in med.items():
  city = m[0]
  counts = m[1]
  t = datetime.datetime.now()
  print(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

In [ ]:
import sqlite3
import datetime

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, masks text, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

# 新增資料

mask = {}

for d in data['features']:
  # 將每一筆資料分別取出來新增到資料庫
  city = d['properties']['county']
  if city == "":
    city = d['properties']['address'][:3]
  if city not in mask:
    mask[city] = 0
  maskCounts = d['properties']['mask_adult'] + d['properties']['mask_child']
  mask[city] += maskCounts
for city, counts in mask.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO masks VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())

conn.commit()
conn.close()